# LAB 3: Snow Cover Analysis and SWE Reconstruction

In this lab we will look at high resolution reserach grade snow cover and SWE products for one of the AOI basins. In additon to the Snow water equivalent, a main theme of these trainings, we will also look at the snow covered area. Snow covered area is far easier to accuratly measure in near real time,can give useful clues to water supplies from seasonal snow, and contextualizes the current year. Our view is that alongisde SWE information,  high quality snow cover maps have a large roll to play in decision support tools for water management in snow domination basins.  
  
For this lab, we will be looking at 7 years of Snow Covered Area and Snow Water Equivalent data at 500m spatial resolution for the Panjsher basin in Afganistan. The snow covered area data is derived from MODIS satellite data and the Snow Water Equivalent data is generated using SWE reconstruciton of the MODIS snow covered area data combined with energy balance estimats. In lecture # 3 we give a detailed account of how these datasets are created. This lab is focsed on using these datasets and understand how they can be used in decision support.  

In [ ]:
#set up the lab workspace
import geopandas as gpd 
import rasterio as rio
from rasterio.plot import show
import numpy as np
import pandas as pd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import copy
from datetime import date
import contextily as ctx
%matplotlib inline

# MODIS datasets
This lab uses data from MODIS, the Moderate Resolution Imageing Spectrometer. MODIS images almost the whole earth everyday. A great way to see an example of what MODIS data look like is to browse NASA worldview:  

https://worldview.earthdata.nasa.gov/  


Browsing Worldview you can see the swaths of the images MODIS aquires onboard the AQUA and TERRA satellites. Browse and look for snow cover in the mountains each day. Some questions to help you think about estimating snow with optical satellites:  

- Can you find a mountainous region you are interested in the snowpack of on Worldview?
- Can you see all the snow each day?
- Are there clouds in the way often?
- How does the cloud cover obsurcing the snow vary by time of year? By location in High Mountain Asia?

It is always good to look at the "raw" data that we generate our snow products from to improve your understanding of how we get to the final estimates of snow properties. This helps develop your understanding of the strengths and limitations of various apporaches to estiamting SWE, as we have mentioned in prior labs. 

For SWE and snow cover from MODIS, we have to remember that we are relying on an optical satellite for the two crutial snow measuremnts we need for reconsturciton:

- Snow covered area
- Snow albedo (via grain size and contaminat concentration) 

As lecture 3 goes into detail on, these are two critical measurements that we need from MODIS to drive the SWE reconstrution. 


## MODIS data and the sinusodial projection
Another important point about MODIS is that data are in the sinusodial- equal area projection from the MODIS satellite. You will see in this lab, that we have left the data in the standard MODIS projection. The basin shape may look different than you expect it to, that is due to the projection of the data. Best practice is to perform all calculations on the data in the projection that it is delivered in, or in the raw satellite swath level format. Everytime the data is reprojected, there is informaton loss and the quality of the estimates decrease. We stringly recommend that users perform all MODIS snow analysis in the sinusodial projection the data is delivered in, and then reproject final maps for qualitativle visualization only in whichever projection end users prefer.

Here are some resources on the MODIS sensor that we will be using:

First, there are two MODIS sensors in space, on two different satellites. 

TERRA (morning MODIS images each day): https://terra.nasa.gov/about/terra-instruments/modis

AQUA (afternoon MODIS images each day): https://oceancolor.gsfc.nasa.gov/data/aqua/

Aqua suffered a loss of band 6 early on. This band is crutial for estimating the albedo of the snow. The loss of band 6 and the fact that aqua aquires images in the afternoon, when convection in the atmosphere tends to make mountainous regions more cloudy, means that we do not use AQUA MODIS data for our SWE analysis. We instead only use the TERRA datasets as they are higher quality and have less cloud cover issues. 

General information about MODIS: https://modis.gsfc.nasa.gov/about/    

Information on the MODIS Grid: https://modis-land.gsfc.nasa.gov/MODLAND_grid.html  

Informaton on the Sinusoidal Projection: https://en.wikipedia.org/wiki/Sinusoidal_projection  

For this lab, we have prepared snow cover and snow water equivalent data from MODIS for 2003-2011 to explore how these data can be useful for water supply management. 

### Visualize the basin we will explore snow cover and SWE within for the lab

In [ ]:
#lets look at the Area of Interest (AOI) on a map 
# to remind us where we will be looking at the data from.
aoiFile='panjsher_basin_afghanistan_wgs84.shp'

#load the shapefile of the basin boundary
AOI = gpd.read_file(aoiFile)
AOI.crs= "EPSG:4326" #({'init': 'epsg:4326'})

#plot - set the plot to show a greater extent around the basin than we saw in Lab 1
fig,ax=plt.subplots(figsize=(15,15))
AOI.plot(edgecolor="purple", facecolor="None", ax=ax, linewidth=4)
plt.xlim([67, 72]) #longitude
plt.ylim([33, 38]) #latitude
ctx.add_basemap(ax, zoom=7, crs="EPSG:4326");

# Logical Basin Mask - how we calcualte estimates for a specific basin

Logical masks are a power tool for analysizing raster data. Instead of a shapefil like we see above, which is a vector line around the bains, a logical mask is a raster that is the same extent as the underlying remote sensing data, and says exactly which pixels are in or out of the basin, we can use this logical mask to color our visualizaing in useful ways, perform calculatons on the SWE and snow cover data, and focus our anaysis on the basin of interest. Below is an image of the mask we will use for this basin. On the backend, when we do analysis of individual regions or what to constrain our analysis to specific subsets of pixels, we use logical masks to perfrom these selections. 

## In the code below, how do the numbers represent which pixels are within the basin and which pixels are outside of the basin?

In [ ]:
#load the basin mask
basinMaskFile='panjsher_basin_afghanistan_basinMask.tif'
bMask=rio.open(basinMaskFile)
basinMask = bMask.read()
basinMask=np.squeeze(basinMask)
fig,ax=plt.subplots(figsize=(15,15))
#plot the fsca image and the correct colormap with maxcolor value set to fsca=100
im=ax.imshow(basinMask)
#plt.show()
#add the colorbar
fig.colorbar(im,ax=ax)

# Fractional Snow Covered Area: Why fractions/percent cover? 


Why do we need an estimate of the fractional cover of snow wihtin each pixel? This is very important for remote sening of seasonal snow in the mountains.   
  
##### Snow varies at a finer spatial resolution than the size of the pixels in the remote sensing images we have of the snow  
  
Because of this, a binary map of snow for each pixel (snow/no snow) is insufficent. Becuase the snow can cover just a small portion of a pixel, or a large portion of a pixel, and becuase the percent cover of a pixel can change over the course of many weeks as the snow melts, we need to understand exactly what fraction of the pixel is covered in snow for our snow cover and SWE analysis work. 

Here is a good paper explaining the prevalence of mixed pixels in the mountains and the need to use fractional snow cover estiamtes: https://doi.org/10.3390/rs61212478

### Lets look at Figure 6 and Figure 7 in the above paper to see why we need to estimate the percent cover of snow within each MODIS pixel. 

Now that we have an understanding of the need for fracitona snow cover estimates, lets look at maps of snow cover an then the corresponding maps of snow water equivalent.   

# Maps of Snow Covered Area
For this section of the lab we will visualize maps of fractional snow covered area (also refered to as snow cover percent) Let's pick a year of data to explore first. 

In [ ]:
#pick a calendar year to explor fractional snow covered area from
calendarYear='2011'

#file with the fsca maps for the calendar year of interest
fname='panjsher_basin_afghanistan_fsca_'+calendarYear+'.tif'

#load that year of fractional snow cover data with rasterio
#open the file with rasterio
src=rio.open(fname)

#load the image data into our workspace with rasterio
FSCA = src.read()

#see how big the dataset is
numel=FSCA.shape
print(numel)

# Look at one day of Fsca data

To start our exploration of these data, lets visualize one day of fsca data and see what it looks like. An imporant part of visualization is choosing good colors for maping the raster data. matplotlib offers great default colormaps. For future work it is always good to spend some time thinking about how you want to present the data. The colors that are choosen can have a signifcant impact on how users respond to the data. 


# Choosing a colormap

Look at the default options in matplot lib: 
https://matplotlib.org/stable/tutorials/colors/colormaps.html

## QUESTION: What are six general catagoires of colormaps for various types of data? 


# useful resource for picking colormaps

The link below is a great tool to help figure out possible colormaps to use for various datasets:

https://colorbrewer2.org

Things to consider when choosing how to visualize data:
- colorblindness
- sequential, diverging, or qualitative values
- one huse vs multi hue
- does the colormap work when printed in black and white?
- contextual information colors and inclusion

# Visualize the snow cover data with a good colormap

In [ ]:
# pick a day of year to visualize snow cover percent from 
doy=11

set the colormap - we will use the "Blues" colormap

In [ ]:
cmap=copy.copy(plt.get_cmap("Blues"))

reverse the standard blue colormap - we want "white" colors to represent high fsca values 
and "blue" colors to represent low to no fsca pixels

In [ ]:
cmap=cmap.reversed()

nans are set to the maximum integer value in an 8 bit unsigned integer dataset ( a value of 255) 
we do not want to color them blue, we want them not included in the colormap, we will set them to grey
a different color that contrasts with our choosen colormap.

In [ ]:
cmap.set_over('grey')

Now we can plot our map!

In [ ]:
#set up the plot
fig,ax=plt.subplots(figsize=(15,15))
#plot the fsca image and the correct colormap with maxcolor value set to fsca=100
im=ax.imshow(src.read(doy),cmap=cmap,vmax=100)
#plt.show()
#add the colorbar
cbar=fig.colorbar(im,ax=ax)
cbar.set_label(label='snow cover percent',size=20)

# Plot timeseries for one pixel for one year

In [ ]:
# Create figure and plot space
fig, ax = plt.subplots(figsize=(10, 10))

# Add x-axis and y-axis
ax.plot(FSCA[:,100,125],
        color='purple')

# Set title and labels for axes
ax.set(xlabel="Day of Year",
       ylabel="Snow cover fraction of the pixel",
       title="Daily Snow Cover Fraction")

# Rotate tick marks on x-axis
plt.setp(ax.get_xticklabels(), rotation=45)
plt.show()

# Plot basin wide snow cover for each day of the year

We now want to look at the total snow cover of the basin by calculating the basin wide fsca each day. below is our function that performs this calculation:

In [ ]:
def compute_basin_fsca(dataCube,basinMask):
    idx=np.nonzero(basinMask)
    numBasinPxls=np.count_nonzero(basinMask)
    numValid=np.count_nonzero(basinMask)
    numel=dataCube.shape
    output=np.zeros(numel[0])   
    for i in range(numel[0]):
        day_fsca=np.squeeze(dataCube[i,:,:])
        
        #remeber that values above 100 are fill, so we dont want to add (snow cover can only go to 100% cover of the pixel)
        day_fsca[day_fsca>100]=0
        
    
        #fraction of basin covered by snow each day     
        output[i]=day_fsca[idx].sum()/numBasinPxls
    return output

dailyBasinSnowCover=compute_basin_fsca(FSCA,basinMask)
dailyBasinSnowCover.shape

Now we can take the output and plot it on a graph

In [ ]:
# Create figure and plot space
fig, ax = plt.subplots(figsize=(10, 10))

# Add x-axis and y-axis
ax.plot(dailyBasinSnowCover,
        color='purple')

# Set title and labels for axes
ax.set(xlabel="Day of Year",
       ylabel="Snow cover fraction (%)",
       title="Daily Snow Cover Fraction of the Entire Basin")

# Look at basin wide snow cover percent each calendar year

What if we want to look at all the years at the same time? This can be very useful for contextualizing snow informaton. The plot below is of all years basin wide snow cover overlaid on top of each other. 

In [ ]:
#Create figure and plot space
plt.figure(figsize=(15,10));

#plot all the background years
yrs=list(range(2003,2012))

for i in range(2003,2012):
  
    #load the data from the current year
    tmpName='panjsher_basin_afghanistan_fsca_'+str(i)+'.tif'
    #load the year of fractional snow cover data
    tmpSrc=rio.open(tmpName)
    tmpFSCA=tmpSrc.read()
    
    tmp_dailyBasinSnowCover=compute_basin_fsca(tmpFSCA,basinMask)

    
# Add x-axis and y-axis - x axis is not year this tim ebecuase we have lots of years overlaied
    plt.plot(tmp_dailyBasinSnowCover,
            color='grey')

#highlight the 2004-2005 water year
#plt.plot(list(range(len(pointWY_era5))),
#         pointWY_era5,
#         color='purple')

# Set title and labels for axes
titleText='Daily Basin Wide Snow Cover \n' + 'Panjsher Basin Afghanistan'
plt.title(titleText);
plt.xlabel("Calendar Year Days");
plt.ylabel("basin wide snow cover (%)");

# Look at basin wide snow cover percent each water year

calendar years are not the best frame of view for looking at snow cover and swe, lets rearrange the data to look at it by water year instead. 

In [ ]:
#Create figure and plot space
plt.figure(figsize=(15,10));

# k will step our index forward
# the loop runs by water year, 
# we want the k index to start at zero and go up by one each loop
k=0
for i in range(2004,2011):
  
    #pick a water year and a day of the year to create the snow covered days map for
    WaterYear=i

    # Specific date
    day_of_year_wy_start = date(WaterYear-1, 10, 1).timetuple().tm_yday
    day_of_year_wy_end = date(WaterYear, 9, 30).timetuple().tm_yday

    #create an empty arrary for snow covered days calculation to go into
    fname1='panjsher_basin_afghanistan_fsca_'+str(WaterYear-1)+'.tif'
    fname2='panjsher_basin_afghanistan_fsca_'+str(WaterYear)+'.tif'

    #load data and create a water year array
    #load the year of fractional snow cover data
    wy1=rio.open(fname1)
    wy2=rio.open(fname2)

    #image data
    fsca1=wy1.read()
    fsca2=wy2.read()

    #merge the correct portion of each calendar years dataset to create the water year datacube
    #days we dont need from calendar year at start of the water year
    idx1=list(range(0,day_of_year_wy_start-1))
    #days we dont need from calendar year at end of the water year
    idx2=list(range(day_of_year_wy_end,365))

    #extract the valid dates from the water year from each calendar year array
    fsca1=np.delete(fsca1,idx1,axis=0)
    fsca2=np.delete(fsca2,idx2,axis=0)
    #concatinate the valid dates into a single water year array
    wy_fsca=np.concatenate((fsca1,fsca2))
      
    tmp_dailyBasinSnowCover=compute_basin_fsca(wy_fsca,basinMask)

   
    #step index forward - the loop runs by year, we want the k index to start at zero and go up by one each loop
    k=k+1

# Add x-axis and y-axis - x axis is not year this time becuase we have lots of years overlaied
    plt.plot(tmp_dailyBasinSnowCover,
            color='grey')

# Set title and labels for axes
titleText='Daily Basin Wide Snow Cover \n' + 'Panjsher Basin Afghanistan'
plt.title(titleText);
plt.xlabel("Calendar Year Days");
plt.ylabel("basin wide snow cover (%)");

# MODIS Fracitonal Snow Cover Decision Support Tools

Now that we have the basin wide and per pixel snow cover through time for each year, we can take this data and turn it into useful information for decision making. Below are a few graphs that help turn snow cover data into actionable information.

# Snow Covered Days

We are going to look at how many snow covered days each pixel has in each water year.

In [ ]:
#pick a water year and a day of the year to create the snow covered days map for
WaterYear=2004

# Specific date
day_of_year_wy_start = date(WaterYear-1, 10, 1).timetuple().tm_yday
day_of_year_wy_end = date(WaterYear, 9, 30).timetuple().tm_yday

#create an empty arrary for snow covered days calculation to go into
fname1='panjsher_basin_afghanistan_fsca_'+str(WaterYear-1)+'.tif'
fname2='panjsher_basin_afghanistan_fsca_'+str(WaterYear)+'.tif'

#load data and create a water year array
#load the year of fractional snow cover data
wy1=rio.open(fname1)
wy2=rio.open(fname2)

#image data
fsca1=wy1.read()
fsca2=wy2.read()

#merge the correct portion of each calendar years dataset to create the water year datacube
#days we dont need from calendar year at start of the water year
idx1=list(range(0,day_of_year_wy_start-1))
#days we dont need from calendar year at end of the water year
idx2=list(range(day_of_year_wy_end,365))

#extract the valid dates from the water year from each calendar year array
fsca1=np.delete(fsca1,idx1,axis=0)
fsca2=np.delete(fsca2,idx2,axis=0)
#concatinate the valid dates into a single water year array
wy_fsca=np.concatenate((fsca1,fsca2))

Now that we have a water year of daily snow covered area assembled, we can do a snow covered days analysis on the data cube. For snow covered days, we are only interested in how many days the pixel was snow covered this water year. So we need to convert the fractional snow cover data into a new variable that has a binary snow mask that identifies each day a pixel has snow cover.

In [ ]:
#snow cover logical mask is any pixel where snow cover is greater than 0%, and not any of the fill pixels (fsca>100)
snow_cover=(wy_fsca>0)&(wy_fsca<=100)

Now that we have this mask, we can look at it. This is similar to what a buinary snow cover product would look like. While it is not as useful as fractional snow cover for a daily view, we can add up the total number of days of snow cover and gain useful informaton about the basin. Another important point is that binary snow maps usually cannot reliabaly identify snow cover for pixels that are less than 50% snow covered so our snow cover map from fractional snow cover data is more extensive than a traditional binary snow cover map becuase the fractional snow cover can be detected when the snow cover is as low at 10% cover for a pixel. 

In [ ]:
#View one days of snow cover
#set up the plot
fig,ax=plt.subplots(figsize=(15,15))
#plot the fsca image and the correct colormap with maxcolor value set to fsca=100
im=ax.imshow(snow_cover[10,:,:])
#plt.show()
#add the colorbar
cbar=fig.colorbar(im,ax=ax)

### Compare the binary map above to the fractional snow cover map earlier in the lab, see how much more information is in the fractional snow cover map.  

# Plot Snow Covered Days
Now that we see what snow cover looks like, we can add up how many days so far in the water year each pixel has had snow cover. To do this, we need to pick a day of the year that we want to see the snow covered days estimate for. 
## Convert a date to a day of water year date
To do this we will take a date ( month and day of month) and convert it to a dayof water year date. This is the number of the day within the water year that we want to see the snow covered days estimate for. We use water year for this analsyis becuase we want to capture the date the pixel became snow covered - which usually occures prior to January first for many pixels, and is also usually some day after the start of the water year. 

In [ ]:
#pick a date we want to see snow covered days for
#month
m=4
#day of month
dom=25
#number of days from prior calendar year in the water year array
numel=fsca1.shape
doy=date(WaterYear, m, dom).timetuple().tm_yday
#day in the water year array we want to look at per pixel snow covered days
analysisDay=numel[0]+doy-1
print('the calendar date '+ str(m) + '-' + str(dom) + ' is the water year day: ' + str(analysisDay))

now that we have picked our analysis day and have the index for this day within the water year we can add up the total number of snow covered days for each pixel so far in the water year leading up to the analysis day and visualize the result.

In [ ]:
# fill the array with the snow covered days for each pixel
snow_covered_days=np.count_nonzero(snow_cover[0:analysisDay-1,:,:], axis=0, keepdims=True)

#View one days of snow cover
#set up the plot
fig,ax=plt.subplots(figsize=(15,15))
#plot the fsca image and the correct colormap with maxcolor value set to fsca=100
im=ax.imshow(np.squeeze(snow_covered_days, axis=0))
#plt.show()
#add the colorbar
cbar=fig.colorbar(im,ax=ax)
cbar.set_label(label='snow covered days',size=20)

# Snow Covered Days - compared to the mean snow covered days

Now that we have looked at the snow covered days for one year of data, another useful way to analyize the snow covered days is to look at the relationship between the current year snow covered days and the mean number of snow covered days for that pixel for the period of record. 



In [ ]:
# create an array that is the number of snow covered days for all years for the date of interest. 
#pick a date we want to see snow covered days for:
#month
m=4
#day of month
dom=25

#empty array for 7 years of snow covered day data for the size of our image
snow_covered_days_record=np.empty([7,196,169])
snow_covered_days_record.shape

A common way to loop through data and perfrom calculations is to first "initalize" the variable that will eventually hold the answers to your calculations. We can see from above that we have initalized an empty array that can hold 7 years of snow covered days analysis for our date of interest and that the empty arrary has the same dimesions as the daily snow cover data. Now we will loop through all 7 water years and calculate the number of snow covered days for each pixel each water year.

In [ ]:
# k will step our index forward
# the loop runs by water year, 
# we want the k index to start at zero and go up by one each loop
k=0
for i in range(2004,2011):
  
    #pick a water year and a day of the year to create the snow covered days map for
    WaterYear=i

    # Specific date
    day_of_year_wy_start = date(WaterYear-1, 10, 1).timetuple().tm_yday
    day_of_year_wy_end = date(WaterYear, 9, 30).timetuple().tm_yday

    #create an empty arrary for snow covered days calculation to go into
    fname1='panjsher_basin_afghanistan_fsca_'+str(WaterYear-1)+'.tif'
    fname2='panjsher_basin_afghanistan_fsca_'+str(WaterYear)+'.tif'

    #load data and create a water year array
    #load the year of fractional snow cover data
    wy1=rio.open(fname1)
    wy2=rio.open(fname2)

    #image data
    fsca1=wy1.read()
    fsca2=wy2.read()

    #merge the correct portion of each calendar years dataset to create the water year datacube
    #days we dont need from calendar year at start of the water year
    idx1=list(range(0,day_of_year_wy_start-1))
    #days we dont need from calendar year at end of the water year
    idx2=list(range(day_of_year_wy_end,365))

    #extract the valid dates from the water year from each calendar year array
    fsca1=np.delete(fsca1,idx1,axis=0)
    fsca2=np.delete(fsca2,idx2,axis=0)
    #concatinate the valid dates into a single water year array
    wy_fsca=np.concatenate((fsca1,fsca2))
    
    #snow cover logical mask is any pixel where snow cover is greater than 0%, and not any of the fill pixels (fsca>100)
    snow_cover=(wy_fsca>0)&(wy_fsca<=100)
    
    #number of days from prior calendar year in the water year array
    doy=date(WaterYear, m, dom).timetuple().tm_yday
    #day in the water year array we want to look at per pixel snow covered days
    analysisDay=numel[0]+doy-1
    
    # fill the array with the snow covered days for each pixel
    snow_covered_days=np.count_nonzero(snow_cover[0:analysisDay-1,:,:], axis=0, keepdims=True)

    #fill our array we initalized before the loop
    snow_covered_days_record[k,:,:]=np.squeeze(snow_covered_days, axis=0)
    
    #step index forward - the loop runs by year, we want the k index to start at zero and go up by one each loop
    k=k+1
    
snow_covered_days_record.shape 

After looping through all the data, our array is the same size, but now it is filled with all the data. Now we can view the snow covered days on our day of interest for any water year from our record.  try a few out with the code below.

# Question: Do you see any differences in each water year snow covered days analysis?

In [ ]:
#pick a number 0-6 for which year you want to plot the snow covered days analysis for. 
year2plot=0

#View one days of snow cover
#set up the plot
fig,ax=plt.subplots(figsize=(15,15))
#plot the fsca image and the correct colormap with maxcolor value set to fsca=100
im=ax.imshow(snow_covered_days_record[year2plot,:,:])
#plt.show()
#add the colorbar
cbar=fig.colorbar(im,ax=ax)
cbar.set_label(label='snow covered days',size=20)

# Snow Covered Days - Comparing to the mean

Now we can generate the mean number of snow covered days for each pixel to get the average length of time each pixel is covered by snow from our record. 

In [ ]:
#average number of snow covered days
avg_scd=np.mean(snow_covered_days_record,axis=0)

#plot the average number of snow covered days
#View one days of snow cover
#set up the plot
fig,ax=plt.subplots(figsize=(15,15))
#plot the fsca image and the correct colormap with maxcolor value set to fsca=100
im=ax.imshow(avg_scd)

#add the colorbar
cbar=fig.colorbar(im,ax=ax)
cbar.set_label(label='mean number of snow covered days',size=20)

A useful tool is to then compare the current year to the average to understand how snow cover compares to past years. 

## Question: What type of colormap would be best for this type of comparison?

In [ ]:
#pick a number 0-6 for which year you want to plot the snow covered days analysis for. 
year2plot=0

# difference between current year and average snow covered days
scd_diff=snow_covered_days_record[year2plot,:,:]-avg_scd
#set values outside the basin to fill
scd_diff[basinMask==0]=999

# plot the results

#pick a good colormap for visualzing  the data!
#set the colormap
cmap=copy.copy(plt.get_cmap("PiYG"))
#nans are maxint uint8 (255) we do not want to color them blue, we want them not included in the colormap
cmap.set_over('grey')

#set up the plot
fig,ax=plt.subplots(figsize=(15,15))
#plot the fsca image and the correct colormap with maxcolor value set to fsca=100
im=ax.imshow(scd_diff,cmap=cmap,vmax=40)
#plt.show()
#add the colorbar
cbar=fig.colorbar(im,ax=ax)
cbar.set_label(label='differnece from mean snow covered days',size=20)

# Part 2: SWE Reconstruciton 

We just looked at snow covered area - which we have avalible in real time and helps us understand some of the differences between the current year and past years. Now we will look at SWE reconstruction, which givens us detailed data on past years water stored in the snowpack, and is a useful tool for training machine learning SWE forecasting models.

We will learn how to do the following with SWE reconstructions:

- one year of swe
- plot for one point
- visualize on difffernt days
- show the melt curve for each year overlaid

#### SWE Units: Raw swe data for this lab has been saved in cm to make the files smaller (uint8 data)

# Maps of Snow Water Equivalent
For this section of the lab we will visualize maps of Snow Water Equivalent (SWE) Let's pick a year of data to explore first. 

In [ ]:
#pick a calendar year to explore snow water equivalent
calendarYear='2011'

#file with the fsca maps for the calendar year of interest
fname='panjsher_basin_afghanistan_ParBal_SWE_'+calendarYear+'.tif'

#load that year of fractional snow cover data with rasterio
#open the file with rasterio
src=rio.open(fname)

#load the image data into our workspace with rasterio
SWE = src.read()

#see how big the dataset is
numel=SWE.shape
print(numel)

# Look at one day of SWE data

To start our exploration of these data, lets visualize one day of SWE data and see what it looks like. An imporant part of visualization is choosing good colors for maping the raster data. Refer back to the first section of the lab on colormaps to think about what a good colormap would be for the SWE dataset.

In [ ]:
# pick a day of year to visualize snow cover percent from 
doy=100

set the colormap - we will use the "viridis" perceptually uniform sequential colormap

In [ ]:
cmap=copy.copy(plt.get_cmap("viridis"))

nans are set to the maximum integer value in an 8 bit unsigned integer dataset ( a value of 255) 
we do not want to color them blue, we want them not included in the colormap, we will set them to grey
a different color that contrasts with our choosen colormap.

In [ ]:
cmap.set_under('grey')

Now we can plot our map!

In [ ]:
#load the image to plot and set fill values to zero
plotSWE=src.read(doy)
plotSWE[plotSWE==255]=0
#set up the plot
fig,ax=plt.subplots(figsize=(15,15))
#plot the fsca image and the correct colormap with maxcolor value set to fsca=100
im=ax.imshow(plotSWE,cmap=cmap,vmin=1)
#plt.show()
#add the colorbar
cbar=fig.colorbar(im,ax=ax)
cbar.set_label(label='snow water equivalent (cm)',size=20)

In [ ]:
pointSWE=SWE[:,125,20]
pointSWE[pointSWE==255]=0

# Create figure and plot space
fig, ax = plt.subplots(figsize=(10, 10))

# Add x-axis and y-axis
ax.plot(pointSWE,
        color='purple')

# Set title and labels for axes
ax.set(xlabel="Day of Year",
       ylabel="Snow water equivalent of the pixel (cm)",
       title="Daily Snow water equivalent")

# Rotate tick marks on x-axis
plt.setp(ax.get_xticklabels(), rotation=45)
plt.xlim(80,160)
plt.show()

# Plot basin wide SWE  for each day of the year

We are now going to calculate the total water volume in the snow pack each year in the basin. This is the same calculations we did on AMSR and ERA-5 data in Lab 1

In [ ]:
def compute_volume(dataCube,basinMask):
    idx=np.nonzero(basinMask)
    numValid=np.count_nonzero(basinMask)
    numel=dataCube.shape
    output=np.zeros(numel[0])   
    for i in range(numel[0]):
        daySWE=np.squeeze(dataCube[i,:,:])
        
        #remeber that values of 255 are fill in the original dataset, so we dont want to add - assume missing data
        daySWE[daySWE==255]=0
        
        #volume of water in each pixel - calculate in cubic meters and convert to cubic kilometers
        #SWE*area=volumne of water - we want volume in cubic kilometers
        
        #calculate volume in each pixel
        #convert cm to km and then multiply depth (in km) by area (in km2) to get vol (km3)
        pxlVolSWE=daySWE*1e-5*0.463*0.463
        
        #sum up all pixel volumnes
        output[i]=pxlVolSWE[idx].sum()

    return output

dailyBasinSWE=compute_volume(SWE,basinMask)
dailyBasinSWE.shape

In [ ]:
# Create figure and plot space
fig, ax = plt.subplots(figsize=(10, 10))

# Add x-axis and y-axis
ax.plot(dailyBasinSWE,
        color='purple')

# Set title and labels for axes
ax.set(xlabel="Day of Year",
       ylabel="SWE km3",
       title="Daily Snow Water Storage Volumne of the Entire Basin")
plt.xlim(50,200)

# Look at basin wide SWE each calendar year

Now we will take the melt curve from above and plot each years melt curve on the same graph

In [ ]:
#Create figure and plot space
plt.figure(figsize=(15,10));

#plot all the background years
yrs=list(range(2003,2012))

for i in range(2003,2012):
  
    #load the data from the current year
    tmpName='panjsher_basin_afghanistan_ParBal_SWE_'+str(i)+'.tif'
    #load the year of fractional snow cover data
    tmpSrc=rio.open(tmpName)
    tmpSWE=tmpSrc.read()
    
    tmp_dailyBasinSWE=compute_volume(tmpSWE,basinMask)

    
# Add x-axis and y-axis - x axis is not year this tim ebecuase we have lots of years overlaied
    plt.plot(tmp_dailyBasinSWE,
            color='grey')

#highlight the 2004-2005 water year
#plt.plot(list(range(len(pointWY_era5))),
#         pointWY_era5,
#         color='purple')

# Set title and labels for axes
titleText='Daily Basin Wide Snow Storage volume (km3) \n' + 'Panjsher Basin Afghanistan'
plt.title(titleText);
plt.xlabel("Calendar Year Days");
plt.ylabel("water volume (km3)");
plt.xlim(1,250)

# SWE in a given year compared to the mean SWE

Now that we have looked at the snow covered days for one year of data, another useful way to analyize the snow covered days is to look at the relationship between the current year snow covered days and the mean number of snow covered days for that pixel for the period of record. 


In [ ]:
# create an array that is the number of snow covered days for all years for the date of interest. 
#pick a date we want to see snow covered days for:
#month
m=4
#day of month
dom=1

#get the day of year of the analysis
doy=date(2003, m, dom).timetuple().tm_yday

#empty array for 7 years of swe data for the size of our image
swe_record=np.empty([9,196,169])
swe_record.shape

In [ ]:
#average SWE for this date

#loop through and grab SWE from each year for this date
k=0
for i in range(2003,2012):
    print(i)
    #load the data from the current year
    tmpName='panjsher_basin_afghanistan_ParBal_SWE_'+str(i)+'.tif'
    #load the year of SWE data
    tmpSrc=rio.open(tmpName)
    tmpSWE=tmpSrc.read(doy)
    tmpSWE[tmpSWE==255]=0
    
    #fill our array we initalized before the loop
    swe_record[k,:,:]=tmpSWE

    k=k+1
    
swe_record.shape

#calculate the average swe for each pixel on this date, across the record.
avg_swe=np.mean(swe_record,axis=0)

In [ ]:
#plot the mean swe for this date
#set up the plot
fig,ax=plt.subplots(figsize=(15,15))
#plot the fsca image and the correct colormap with maxcolor value set to fsca=100
im=ax.imshow(avg_swe)

#add the colorbar
cbar=fig.colorbar(im,ax=ax)
cbar.set_label(label='mean swe (cm)',size=20)

A useful tool is to then compare the current year to the average to understand how SWE compares to past years. 

## Question: What type of colormap would be best for this type of comparison?

In [ ]:
# difference between current year and average snow covered days
swe_diff=swe_record[1,:,:]-avg_swe

#set values outside the basin to fill
swe_diff[basinMask==0]=999

# plot the results
#pick a good colormap for visualzing  the data!
#set the colormap
cmap=copy.copy(plt.get_cmap("cividis"))
#nans are maxint uint8 (255) we do not want to color them blue, we want them not included in the colormap
cmap.set_over('white')

#set up the plot
fig,ax=plt.subplots(figsize=(15,15))
#plot the fsca image and the correct colormap with maxcolor value set to fsca=100
im=ax.imshow(swe_diff,cmap=cmap,vmax=50)
#plt.show()
#add the colorbar
cbar=fig.colorbar(im,ax=ax)
cbar.set_label(label='differnece from mean SWE (cm)',size=20)

## We can also look at all the years at once and see the spatial and temporal variabiliy of SWE across the basin

In [ ]:
 #pick a good colormap for visualzing  the data!
#set the colormap
cmap=copy.copy(plt.get_cmap("cividis"))
#nans are maxint uint8 (255) we do not want to color them blue, we want them not included in the colormap
cmap.set_over('white')

#set up the plot
fig,axs=plt.subplots(3,3,figsize=(15,15))

#list of the water years
WY=list(range(2003,2012))
    
k=0
   
for col in range(3):
    for row in range(3):
        
        # difference between current year and average snow covered days
        swe_diff=swe_record[k,:,:]-avg_swe

        #set values outside the basin to fill
        swe_diff[basinMask==0]=999
        
        ax = axs[row, col]
        ax.set_title('Water Year: ' + str(WY[k]))
        
        #vmax and vmin keep the colorbars the same between year, colors are same values each year
        pcm = ax.imshow(swe_diff,cmap=cmap,vmax=50,vmin=-40)
        fig.colorbar(pcm, ax=ax)
        k=k+1
plt.show()

We can do this same sort of annual comparison plot wiht the actuall volume of SWE each year on this date, which is also useful:

In [ ]:
#pick a good colormap for visualzing  the data!
#set the colormap
cmap=copy.copy(plt.get_cmap("viridis"))
#nans are maxint uint8 (255) we do not want to color them blue, we want them not included in the colormap
cmap.set_over('white')

#set up the plot
fig,axs=plt.subplots(3,3,figsize=(15,15))

#list of the water years
WY=list(range(2003,2012))
    
k=0
   
for col in range(3):
    for row in range(3):
        
        # difference between current year and average snow covered days
        yr_swe=swe_record[k,:,:]

        #set values outside the basin to fill
        yr_swe[basinMask==0]=999
        
        ax = axs[row, col]
        ax.set_title('Water Year: ' + str(WY[k]))
        
        #vmax and vmin keep the colorbars the same between year, colors are same values each year
        pcm = ax.imshow(yr_swe,cmap=cmap,vmax=100)
        fig.colorbar(pcm, ax=ax)
        k=k+1
plt.show()